In [1]:
import tensorflow as tf

from tensorflow.keras import models, layers, optimizers, losses, datasets

In [4]:
import random, numpy as np

def CreatePairs(img, digit_indices):
  """
  Create Siamese Pairs
  """
  image_pairs = []
  image_label = []
  total_class = min(len(digit_indices[i]) for i in range(10)) - 1

  for i in range(10):
    # Hi ?
    for j in range(total_class):
      # Hi ?
      pair_one, pair_two = digit_indices[i][j], digit_indices[i][j + 1]
      image_pairs += [[img[pair_one], img[pair_two]]]
      inc = random.randrange(1, 10)
      dn = (i + inc) % 10
      # Hi ?
      pair_one, pair_two = digit_indices[i][j], digit_indices[dn][j]
      image_pairs += [[img[pair_one], img[pair_two]]]
      image_label += [1, 0]

  return np.array(image_pairs), np.array(image_label)

def CreatePairsSet(images, labels):
  """
  Create Siamese Pairs Set
  """
  digit_indices = [np.where(labels == i)[0] for i in range(10)]
  pairs, labels = CreatePairs(images, digit_indices)
  labels = labels.astype('float32')
  return pairs, labels

In [5]:
(train_image, train_label), (test_image, test_label) = datasets.fashion_mnist.load_data()

rescale = lambda img : tf.cast(img, tf.float32) / 255.0

train_image = rescale(train_image)

test_image = rescale(test_image)

train_pairs, train_label = CreatePairsSet(train_image, train_label)

test_pairs, test_label = CreatePairsSet(test_image, test_label)

In [6]:
train_image.shape

TensorShape([60000, 28, 28])

In [7]:
train_pairs.shape

(119980, 2, 28, 28)

In [ ]:
import matplotlib.pyplot as plt

pairs = 8

plt.figure(figsize=(10, 10))

plt.imshow(train_pairs[pairs][0])

plt.colorbar()

plt.grid(False);

In [ ]:
plt.imshow(train_pairs[pairs][1])

plt.colorbar()

plt.grid(False);

In [10]:
from tensorflow.keras import backend as K

def SiameseModel():
  """
  Create Siamese Model
  """
  input = layers.Input((28, 28))

  x = layers.Flatten()(input)
  x = layers.Dense(128, activation='relu')(x)

  x = layers.Dropout(0.1)(x)
  x = layers.Dense(128, activation='relu')(x)

  x = layers.Dropout(0.1)(x)
  final = layers.Dense(128, activation='relu')(x)

  return models.Model(inputs=input, outputs=final)

def EucliDistance(vectors):
  """
  Calculate Euclidean Distance
  """
  i, j = vectors

  sum_square = K.sum(K.square(i - j), axis=1, keepdims=True)

  return K.sqrt(K.maximum(sum_square, K.epsilon()))

def EucliDistanceOutputShape(shapes):
  """
  Calculate Euclidean Distance Output Shape
  """
  shape1, shape2 = shapes

  return (shape1[0], 1)

In [ ]:
base_model = SiameseModel()

In [12]:
input_i = layers.Input((28, 28))

input_j = layers.Input((28, 28))

output_i = base_model(input_i)

output_j = base_model(input_j)

In [ ]:
output_distance = layers.Lambda(EucliDistance, output_shape=EucliDistanceOutputShape)([output_i, output_j])

model = models.Model([input_i, input_j], output_distance)

model.summary()

In [ ]:
def ConstrativeMarginLoss(margin):
  def ConstrativeLoss(y_true, y_pred):
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return (y_true * square_pred + (1 - y_true) * margin_square)
  return ConstrativeLoss

optimizer = optimizers.RMSprop()

model.compile(loss=ConstrativeMarginLoss(margin=1), optimizer=optimizer)

train_input = [train_pairs[:, 0], train_pairs[:, 1]]

test_input = [test_pairs[:, 0], test_pairs[:, 1]]

hist = model.fit(train_input, train_label, epochs=20, batch_size=128, validation_data=(test_input, test_label))